# Python objektově orientované programování

<br>

## Základní pilíře objektově-orientovaného programování

---

1. [Polymorfismus](#Polymorfismus-(-~polymorphism)),
    - [terminologie](#Terminologie),
    - [význam v Pythonu](#Význam-v-Pythonu),
    - [využití v OOP](#Využití-v-OOP),
    - [cvičení 7](#🧠-CVIČENÍ-7-🧠,-Vytvoř-třídy-CsvProcessor-a-TxtProcessor:),
2. [zapouzdření](#Zapouzdření-(~encapsulation)),
    - [bez zapouzdření](#Bez-konceptu-zapouzdření),
    - [privátní objekty](#Privátní-objekty),
    - [gettery, settery](#Gettery,-settery),
    - [cvičení 8](#🧠-CVIČENÍ-8-🧠,-Vytvoř-třídu-AutomobilTesla-a-doplň-následující:),
3. [dědičnost](#Dědičnost-(~inheritence)),
    - [význam slova](#Význam-slova),
    - [jednoduchá dědičnost](#Jednoduchá-dědičnost),
    - [funkce super()](#Funkce-super()),
    - [vícenásobné dělení](#Vícenásobné-dědění),
    - [zřetězené dělení](#Zřetězené-dědění),
    - [method resolution order](MRO-(~Method-resolution-order)),
    - [cvičení 9](#🧠-CVIČENÍ-9-🧠,-Vytvoř-třídu-Auto-a-doplň-následující:),
4. [abstrakce](#Abstrakce),
    - [význam slova](#Význam-pojmu),
    - [v praxi](#V-praxi),
    - [knihovna ABC](#Knihovna-abc),
    - [cvičení 10](#🧠-CVIČENÍ-10-🧠,-Vytvoř-třídu-HerniPostava-a-doplň-následující:).

Ve skutečnosti jde o **čtyři teoretické základy**, na kterých OOP stojí.

<br>


## Polymorfismus ( ~polymorphism)

---

<img src="https://i.imgur.com/COUOylr.png" width="1000" style="margin-left:auto; margin-right:auto">

### Terminologie

---

**Polymorfismus** znamená *mnohotvárnost*.

Lidsky si pod tímto můžeš představit situaci, kdy lze jednu věc popsat různými aplikacemi.

Například **pozdrav**. Slušný pozdrav můžeš vyjádřit několika způsoby, nebo dokonce různými jazyky.

<br>

### Význam v Pythonu

---

Ve světě programování  to zase znamená, že jeden objekt, může plnit různé účely:

In [ ]:
print(
    1 + 1,            # operátor '+' s celočíselnými operandy,
    "Lekce" + " #02", # operátor '+' se stringovými operandy,
    sep="\n"
)

<br>

Tento koncept už znáš.

Operátor `+` totiž znamená **pro dva odlišné datové typy, dvě odlišné procedůry**:
1. `int`, sčítání,
2. `str`, spojování (~concatenation).

<br>

Takovou mnohotvárnost, ale neznáš jen mezi `int` a `str`.

Podobného chování si můžeš všimnout i pro uživ. funkci `len`:

In [ ]:
print(
    len("Matous"),                                                             # str
    len(["město", "moře", "kuře", "stavení"]),                                 # list
    len({"jmeno": "Matous", "prijmeni": "Holinka", "email": "matous@nic.cz"}), # dict
    sep="\n"
)

In [ ]:
"Matous".__len__()

<br>

Vidíš, že funkce `len` umí pracovat s různými datovými typy.

<br>

Vrací specifickou hodnotu pro každý datový typ zvlášť. Ale pořád používáš jednu a tutéž funkci.
1. `str`, délka řetězce,
2. `list`, počet údajů v sekvenci,
3. `dict`, počet klíčů v objektu

<br>

### Využití v OOP

---

Dále tento koncept umožňuje přepisovat jména a použití **metod**:

In [ ]:
class CsvProcessor:
    def precti_soubor(self) -> str:
        return "Otevírám CSV soubor..."

In [ ]:
class JsonProcessor:
    def precti_soubor(self) -> str:
        return "Otevírám JSON soubor..."

In [ ]:
csv_soubor = CsvProcessor()
json_soubor = JsonProcessor()

In [ ]:
for soubor in (csv_soubor, json_soubor):
    print(soubor.precti_soubor())

<br>

Prakticky to tedy znamená, že *polymorfismus* umožní objektům **různých tříd**, pracovat **se stejnojmennými metodami**.

S dalším využitím *polymorfismu* se můžeš setkat u dědičnosti (později v materiálech).

<br>

### 🧠 CVIČENÍ 7 🧠, Vytvoř třídy `CsvProcessor` a `TxtProcessor`:
---

- Definuj třídu `CsvProcessor`, s metodou `__init__`, která potřebuje jen parametr `jmeno_souboru`,
- zadej třídě `CsvProcessor` třídní atribut `ext` s hodnotou `.csv`,
- metoda `__init__` sama ověří, jestli proměnná obsahuje správnou příponu,
- pokud je přípona `.csv`, definuj instanční atribut, jinak vyvolej výjimku s textem: `Špatný formát souboru`,
- vytvoř metodu `nacti_obsah`, která přečte a vrátí obsah **csv** souboru,
- definuj třídu `TxtProcessor`, s metodou `__init__`, která potřebuje jen parametr `jmeno_souboru`,
- zadej třídě `TxtProcessor` třídní atribut `ext` s hodnotou `.txt`,
- metoda `__init__` sama ověří, jestli proměnná obsahuje správnou příponu,
- pokud je přípona `.txt`, definuj instanční atribut, jinak vyvolej výjimku s textem: `Špatný formát souboru`,
- vytvoř metodu `nacti_obsah`, která přečte a vrátí obsah **txt** souboru.

Před spuštěním ukázky si nezapomeň vytvořit pokusné soubory.

In [ ]:
csv_1 = CsvProcessor("csv_pokus.csv")
txt_1 = TxtProcessor("txt_pokus.txt")

<details>
    <summary>▶️ Řešení</summary>
    
    ```
    from pathlib import Path

    from pandas import read_csv
    
    
    class CsvProcessor:
        ext: str = ".csv"

        def __init__(self, jmeno_souboru: str):
            if Path(jmeno_souboru).suffix != self.ext:
                raise Exception("Špatný formát souboru")
            self.jmeno_souboru = jmeno_souboru

        def nacti_obsah(self):
            return read_csv(self.jmeno_souboru)


    class TxtProcessor:
        ext: str = ".txt"

        def __init__(self, jmeno_souboru: str):
            if Path(jmeno_souboru).suffix != self.ext:
                raise Exception("Špatný formát souboru")
            self.jmeno_souboru = jmeno_souboru

        def nacti_obsah(self):
            with open(self.jmeno_souboru) as txt:
                return txt.read()
    ```
</details>

<br>

## Zapouzdření (~encapsulation)

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.uDpIgfGQDzGDPvxtzsz-5AHaHa%26pid%3DApi&f=1&ipt=eb0bca94ab317533d3313fe92c84884bff2b45c5ce18cfdb8341427e8a8311fe&ipo=images" width="300" style="margin-left:auto; margin-right:auto">

### Terminologie

---

Tento pojem obecně označuje **nějaké skrývání**.

Jde především o práci **s privátními a chráněnými** atributy nebo metodami.

Účelem tohoto skrývání, nebo zapouzdření je ochrana.

<br>

Aby programátoři nepřepisovali, co nemají a tím nezpůsobili **komplikace** zdrojového kódu.

### Bez konceptu zapouzdření

---

V kombinaci s **chráněnými objekty** chceš některé procesy zajistit a současně nekomplikovat uživatelské použití:

In [ ]:
class MojeAplikace:
    
    def __init__(self, jmeno: str, kredit: int):
        self.jmeno = jmeno
        self.kredit = kredit
        
    def vypis_status(self):
        return f"Uživatel: {self.jmeno}, dostupné kredity: {self.kredit}"

In [ ]:
uzivatel_matous = MojeAplikace("Matouš", 100)

In [ ]:
print(uzivatel_matous.vypis_status())

<br>

V aktuálním stavu třídy může **kdokoliv přistup z venku** naší třídy `MojeAplikace` a nedopatřením přepsat hodnoty:

In [ ]:
uzivatel_matous.kredit = 10_000

In [ ]:
print(10_000)

In [ ]:
print(uzivatel_matous.vypis_status())

In [ ]:
uzivatel_matous.kredit = "10_000"

In [ ]:
print(uzivatel_matous.vypis_status())

### Privátní objekty

---

<br>

Proto je vhodné, zápis doplnit:
1. **privátními** `__jmeno`,
2. **chráněnými** `_jmeno`.

In [ ]:
class MojeAplikace:
    
    def __init__(self, jmeno: str, kredit: int):
        self.jmeno = jmeno
        self.__kredit = kredit
        
    def vypis_status(self):
        return f"Uživatel: {self.jmeno}, dostupné kredity: {self.__kredit}"

In [ ]:
uzivatel_lukas = MojeAplikace("Lukáš", 1000)

In [ ]:
print(uzivatel_lukas.vypis_status())

In [ ]:
uzivatel_lukas.kredit = 1_000_000

In [ ]:
print(uzivatel_lukas.vypis_status())

In [ ]:
uzivatel_lukas.__dict__

<br>

Koncept, na kterém je *privátní objekt* v Pythonu postavený, se označuje *name mangling* (tedy *komolení* jmen objektů).

Ale takové opatření pořád neřeší problém, pokud instance dostane třeba **nevhodný datový typ**:

In [ ]:
uzivatel_lukas._MojeAplikace__kredit = "1_000_000"

In [ ]:
print(uzivatel_lukas.vypis_status())

### Gettery, settery

---

<br>

Pro úplný pořádek je ještě nutné stanovit formu, kterou bude atribut zadáván:

In [ ]:
class MojeAplikace:
    
    def __init__(self, jmeno: str):
        self.jmeno = jmeno
        self.__kredit = 0
        
    @property
    def kredit(self):
        """Getter metoda"""
        return self.__kredit
    
    @kredit.setter
    def kredit(self, hodnota: int) -> None:
        """Setter metoda"""
        if isinstance(hodnota, int):
            self.__kredit = hodnota
        else:
            raise Exception("Zadaná hodnota není celé číslo")
        
    def vypis_status(self):
        return f"Uživatel: {self.jmeno}, dostupné kredity: {self.__kredit}"

In [ ]:
uzivatel_lukas = MojeAplikace("Lukáš")

In [ ]:
uzivatel_lukas.kredit = 1000

In [ ]:
print(uzivatel_lukas.vypis_status())

<br>

Nyní není možné zadat jiný datový typ, než celé číslo:

In [ ]:
uzivatel_lukas.kredit = "1000"

<br>

Pomocí defaultní výjimky `Exception` nyní uživatel vidí, že není možné pracovat s jiným datovým typem.

#### Souhrn k zapouzdření
* **bezpečnost**, u privátních objektů je navíc vrstva ochrany proti nechtěnému přepsaní, nebo použití nesprávného datového typu,
* **kontrola**, ostatní programátoři musí dodržovat postupy, pro manipulaci s privátními objekty,
* **jednoduchost**, rozdělování zadávání a získání hodnot jsou oddělené objekty,
* **čitelnost**, spojování objektů k příslušným třídám dělá zápis přehlednější.

<br>

### 🧠 CVIČENÍ 8 🧠, Vytvoř třídu `AutomobilTesla` a doplň následující:

- Definuj třídu `AutomobilTesla`, s metodou `__init__`, která potřebuje jen parametr `uzivatel` a `cislo`,
- vytvoř **getter a setter** metody pro atribut `max_rychlost`,
- omez zadávání `max_rychlost` jenom pro datový typ `int` z intervalu `0 - 320`,
- pokud uživatel nedodrží podmínky, vyvolej výjimku s textem: `Nepřijatelný datový typ nebo hodnota (< 320 km/h)`,
- vytvoř **getter a setter** metody pro atribut `max_dojezd`,
- omez zadávání `max_dojezd` jenom pro datový typ `int` z intervalu `0 - 590`,
- pokud uživatel nedodrží podmínky, vyvolej výjimku s textem: `Nepřijatelný datový typ nebo hodnota (< 600 km)`,
- přepiš magickou metodu `__str__`, aby formátovala tento výstup: `Uživatel: <uzivatel>, sériové číslo: <cislo>`.

In [ ]:
class AutomobilTesla:
    def __init__(self, uzivatel: str, cislo: int):
        self.cislo = cislo
        self.uzivatel = uzivatel
        self.__max_rychlost = 200
        
    @property
    def max_rychlost(self) -> int:
        print("GETTER: vypisuji rychlost.")
        return self.__max_rychlost
    
    @max_rychlost.setter
    def max_rychlost(self, hodnota: int):
        if isinstance(hodnota, int) and 0 < hodnota < 320:
            self.__max_rychlost = hodnota
            print("SETTER: nastavuji rychlost.")
        else:
            raise Exception("Nepřijatelný datový typ nebo hodnota (< 320 km/h)")

In [ ]:
tesla_model_s = AutomobilTesla("Matouš", "1234ABCE5678")

In [ ]:
tesla_model_s.__dict__

In [ ]:
tesla_model_s.max_rychlost = 250

In [ ]:
tesla_model_s.max_rychlost

In [ ]:
tesla_model_s.__dict__

<details>
    <summary>▶️ Řešení</summary>
    
    ```
    class AutomobilTesla:
        def __init__(self, uzivatel: str, cislo: int):
            self.uzivatel = uzivatel
            self.cislo = cislo

        @property
        def max_rychlost(self):
            return self.__max_rychlost

        @max_rychlost.setter
        def max_rychlost(self, hodnota: int):
            if isinstance(hodnota, int) and 0 < hodnota < 320:
                self.__max_rychlost = hodnota
            else:
                raise Exception("Nepřijatelný datový typ nebo hodnota (< 320 km/h)")

        @property
        def max_dojezd(self):
            return self.__max_dojezd

        @max_dojezd.setter
        def max_dojezd(self, hodnota):
            if isinstance(hodnota, int) and 0 < hodnota < 590:
                self.__max_dojezd = hodnota
            else:
                raise Exception("Nepřijatelný datový typ nebo hodnota (< 600 km)")


        def __str__(self):
            return f"Uživatel: {self.uzivatel}, sériové číslo: {self.cislo}"
    ```
</details>

<br>

## Dědičnost (~inheritence)

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.aD0kgi7jqG8lF8qMmg7Y9QHaFU%26pid%3DApi&f=1&ipt=8e4df2fb68503c0bea5ff03f29e9ffa108168daeaa6dd66662ab080b7951866e&ipo=images" width="300" style="margin-left:auto; margin-right:auto">

### Význam slova

---

Obecně řečeno, **dědictví** znamená nějaký odkaz nebo pozůstatek.

V OOP **Dědičnost** je prvek, díky kterému můžeš přenášet **atributy** a **metody** jedné třídy (*rodičovské*) na další (*potomky*).

V podstatě je každá uživatelem definovaná třída potomkem třídy `object`:

In [ ]:
class MojeTrida(object):
    pass

In [ ]:
print(issubclass(MojeTrida, object))  # object --> MojeTrida --> ...

<br>

V tomto ohledu si ale dědičnosti nevšimneš, protože **interpret** dovoluje rodiče nechat defaultně:

In [ ]:
class Zamestnanec:
    pass

In [ ]:
print(issubclass(Zamestnanec, object))

<br>

Ne vždy se ti hodí **defaultní/standardní** dědičnost.

Třeba pokud potřebuješ vytvořit **vlastní třídy** s vlastními rodiči:

In [ ]:
class Zamestnanec:
    """Objekt pro vytvoření řadového zaměstnance."""
    
    def __init__(self, jmeno: str, vek: int, mzda: int):
        self.jmeno = jmeno
        self.vek = vek
        self.mzda = mzda
        
    def vytvor_email(self, domena: str) -> str:
        self.email = f"{self.jmeno.lower()}{domena}"
        
    # @classmethod
    # def from_string(cls):
    #     pass

In [ ]:
matous = Zamestnanec("Matouš", 40, 80_000)

In [ ]:
matous.vytvor_email("@superdomena.cz")

In [ ]:
print(
    matous.jmeno,
    matous.email,
    sep="\n"
)

<br>

Máš třídu, která řídí proces tvoření nových instancní pro zaměstnance.

Co, když ale potřebuješ další třídu, tentokrát pro *testery*:

In [ ]:
class Tester:
    """Objekt pro vytvoření zaměstnance, testera."""
    
    def __init__(self, jmeno: str, vek: int, mzda: int):
        self.jmeno = jmeno
        self.vek = vek
        self.mzda = mzda
        self.__pristup_vcs = False

    def vytvor_email(self):
        self.email = f"{self.jmeno.lower()}{domena}"
    
    @property
    def pristup_vcs(self):
        return self.__pristup_vcs
    
    @pristup_vcs.setter
    def pristup_vcs(self, hodnota: bool):
        if hodnota in {True, False}:
            self.__pristup_vcs = hodnota
        else:
            raise Exception("Nelze přiřadit takovou hodnotu")

In [ ]:
petr = Tester("Petr", 45, 100_000)

In [ ]:
petr.pristup_vcs = True

In [ ]:
petr.pristup_vcs

### Jednoduchá dědičnost

---

<br>

Na první pohled ale vidíš, že mít **dvě podobné třídy** pod sebou není efektivní a oponuje to princip vývoje softwaru (DRY).

<br>

V rámci specifické pozice bylo potřeba hodně přepisování.

Velkou část třídy `Tester` opakuješ definici třídy `Zamestnanec`.

Z takového důvodu můžeš prakticky použít další, třetí koncept, na kterém OOP stojí. Dědičnost:

In [ ]:
class Tester(Zamestnanec):
    """Objekt pro vytvoření zaměstnance, testera."""
    
    @property
    def pristup_vcs(self):
        return self.__pristup_vcs
    
    @pristup_vcs.setter
    def pristup_vcs(self, hodnota: bool):
        if hodnota in {True, False}:
            self.__pristup_vcs = hodnota
        else:
            raise Exception("Nelze přiřadit takovou hodnotu")

In [ ]:
petr = Tester("Petr", 45, 100_000)

In [ ]:
petr.pristup_vcs = True

In [ ]:
petr.pristup_vcs

In [ ]:
petr.__dict__

In [ ]:
petr.vytvor_email("@superdomena.cz")

In [ ]:
petr.email

<br>

Takovým způsobem můžeš přepisovat méně zápisu a využít již existují, rodičovské třídy.

Pomocí zabudované funkce můžeš potvrdit, že třída `Tester` je potomkem třídy `Zamestnanec`:

In [ ]:
print(issubclass(Tester, Zamestnanec))

### Funkce super()

----

<br>

Co když budeš chtít **upravit stávající objekt u rodiče**:

In [ ]:
class Rodic:
    def funkce_x(self, x):
        print("Rodic", self, x)

In [ ]:
class Potomek(Rodic):
    def funkce_x(self, x):
        print("Potomek", self, x)
        print("Rodic", self, x)  # chci zabránit dalšímu přepisování
        print("Konec tř. Potomek")

In [ ]:
p = Potomek()
p.funkce_x(13)

<br>

U třídy `Potomek` si můžeš povšimnout shodných ohlášení.

Současně ale obsahuje také svoje vlastní.

Tento zápis můžeš upravit pomocí funkce `super()`:

In [ ]:
class Rodic:
    def funkce_y(self, x):
        print("Rodic", self, x)

In [ ]:
class Potomek(Rodic):
    def funkce_y(self, x):
        print("Potomek", self, x)
        super().funkce_y(x)  # __init__()
        print("Konec tř. Potomek")

In [ ]:
p = Potomek()
p.funkce_y(13)

<br>

Funkce `super()` automaticky odkazuje na posledního předchozího rodiče.

Tím se stane zápis méně upovídaný a méně náchylný na chyby.

U funkce `super()` není potřeba doplňovat parametr `self` ani `cls` (interpret doplní automaticky).

<img src="inheritance.png" width="1000" style="margin-left:auto; margin-right:auto">

### Vícenásobné dědění

---

<img src="possibilities.png" width="1000" style="margin-left:auto; margin-right:auto">

Dědění nemusí být jenom jednorázové.

Je možné dědit od jednoho rodiče na několik potomků:

In [ ]:
class Zamestnanec:
    """Objekt pro vytvoření řadového zaměstnance."""
    
    def __init__(self, jmeno: str, vek: int, mzda: int):
        self.jmeno = jmeno
        self.vek = vek
        self.mzda = mzda
        
    def vytvor_email(self, domena: str) -> str:
        self.email = f"{self.jmeno.lower()}{domena}"

In [ ]:
class Tester(Zamestnanec):
    """Objekt pro vytvoření zaměstnance, testera."""
    
    @property
    def pristup_vcs(self):
        return self.__pristup_vcs
    
    @pristup_vcs.setter
    def pristup_vcs(self, hodnota: bool):
        if hodnota in {True, False}:
            self.__pristup_vcs = hodnota
        else:
            raise Exception("Nelze přiřadit takovou hodnotu")

<br>

Navíc přidej další třídu, která se jmenuje `BigDataInzenyr` a stejně jako `Tester` dědí objekty od `Zamestnanec`:

In [ ]:
class BigDataInzenyr(Zamestnanec):
    """Objekt pro vytvoření zaměstnance, big data inženýra."""
    
    @property
    def pristup_db(self):
        return self.__pristup_db
    
    @pristup_db.setter
    def pristup_db(self, hodnota: bool):
        if hodnota in {True, False}:
            self.__pristup_db = hodnota
        else:
            raise Exception("Nelze přiřadit takovou hodnotu")

In [ ]:
filip = BigDataInzenyr("Filip", 40, 120_000)

In [ ]:
filip.pristup_db = True

In [ ]:
filip.vytvor_email("@gmail.com")

In [ ]:
filip.__dict__

<br>

V tomto okamžiku pracuje potomek `BigDataInzenyr` jen se svými metodami, případně s metodami jeho rodiče.

Co když budeš chtít pracovat s metodami **svého sourozence**:

In [ ]:
filip.pristup_vcs = True

In [ ]:
filip.pristup_vcs

<br>

Jak je to možné? Že sourozenci vidí na svoje objekty:

In [ ]:
filip.ma_dovolenou = True

In [ ]:
filip.__dict__

In [ ]:
print(
    issubclass(BigDataInzenyr, Zamestnanec),
    issubclass(Tester, Zamestnanec),
    issubclass(BigDataInzenyr, Tester),
    sep="\n"
)

<br>

Zápisem výše v podstatě vytvoříš nový instanční atribut.

To samozřejmě nemá význam a navíc můžeš uškodit funkcionalitě tříd.

### Zřetězené dědění

---

<img src="chain_inh.png" width="1000" style="margin-left:auto; margin-right:auto">

Narozdíl od vícenásobného dědění, bývá zřetězené málokdy žádoucí.

In [ ]:
class Prarodic:
    def funkce(self, x):
        print("Prarodic", self, x)

In [ ]:
class Rodic(Prarodic):
    def funkce(self, x):
        print("Rodic", self, x)
        super().funkce(x)

In [ ]:
class Potomek(Rodic):
    def funkce(self, x):
        print("Potomek", self, x)
        super().funkce(x)

In [ ]:
p = Potomek()
p.funkce(14)

<br>

Lineární zřetězene dědění je sice možné, ale pomalu přestává být přehledné a únosné.

Pokud potřebuješ využít takový princip, dobře rozmysli, jestli je zápis ještě průhledný a pochopitelný.

<br>

V průběhu času budeš chtít přidat nový typ zaměstnance, opět můžeš aplikovat dědičnost:

In [ ]:
class Zamestnanec:
    """Objekt pro vytvoření řadového zaměstnance."""
    
    def __init__(self, jmeno: str, vek: int, mzda: int):
        self.jmeno = jmeno
        self.vek = vek
        self.mzda = mzda
        
    def vytvor_email(self, domena: str) -> str:
        self.email = f"{self.jmeno.lower()}{domena}"

In [ ]:
class Tester(Zamestnanec):
    """Objekt pro vytvoření zaměstnance, testera."""
    
    @property
    def pristup_vcs(self):
        return self.__pristup_vcs
    
    @pristup_vcs.setter
    def pristup_vcs(self, hodnota: bool):
        if hodnota in {True, False}:
            self.__pristup_vcs = hodnota
        else:
            raise Exception("Nelze přiřadit takovou hodnotu")

In [ ]:
class BigDataInzenyr(Tester):
    """Objekt pro vytvoření zaměstnance, big data inženýra."""
    
    @property
    def pristup_db(self):
        return self.__pristup_db
    
    @pristup_db.setter
    def pristup_db(self, hodnota: bool):
        if hodnota in {True, False}:
            self.__pristup_db = hodnota
        else:
            raise Exception("Nelze přiřadit takovou hodnotu")

In [ ]:
tomas = BigDataInzenyr("Tomáš", 31, 70_000)

In [ ]:
tomas.pristup_db = True

In [ ]:
tomas.pristup_vcs = True

In [ ]:
print(
    tomas.jmeno,
    tomas.pristup_db,
    tomas.pristup_vcs,
    sep="\n"
)

In [ ]:
tomas.__dict__

In [ ]:
tomas.vytvor_email("@gmail.com")

In [ ]:
tomas.__dict__

<br>

Opravdu už není jednoduché, sledovat původ změn.

A v této ukázce jde o triviální třídy.

<br>

Co když se potom dědičnost ještě zkomplikuje?

<img src="https://i.imgur.com/HgQpT9I.png" width="1000" style="margin-left:auto; margin-right:auto">

### MRO (~Method resolution order)

---

Pokud je systému dědičnosti větší počet úrovní, nebylo by snadné sledovat rodiče.

Z takového důvodu existuje metoda, která označí cestu, kterou jsou objekty sbírány:

In [ ]:
class Employee:
    def f(self):
        print("Employee", self)

class FrontendDev(Employee):
    def f(self):
        print("FronendDev", self)
        super().f()

class BackendDev(Employee):
    def f(self):
        print("BackendDev", self)
        super().f()
        
class FullstackDev(FrontendDev, BackendDev):
    def f(self):
        print("FullstackDev", self)
        super().f()

In [ ]:
fdev = FullstackDev()

In [ ]:
fdev.f()

<br>

MRO je také jedna z magických metod, takže můžeš pracovat i pomocí ní:

In [ ]:
FullstackDev.__mro__

<br>

Proč je tedy MRO tak podstatné?

Podívej se na drobnou změnu v zápise:

In [ ]:
class Employee:
    def f(self):
        print("Employee", self)

class FrontendDev(Employee):
    def f(self):
        print("FronendDev", self)
        super().f()

class BackendDev(Employee):
    def f(self):
        print("BackendDev", self)
        # Chybí "super()"
        
class FullstackDev(FrontendDev, BackendDev):
    def f(self):
        print("FullstackDev", self)
        super().f()

In [ ]:
fdev = FullstackDev()

In [ ]:
fdev.f()

<br>

Z toho vyplývá, že `super()` nepracuje přímo s rodičem, nebo sourozence.

Pracuje s předchůdcem, kterého stanoví MRO:

In [ ]:
class Employee:
    atr = "E"

class FrontendDev(Employee):
    atri = "Fr"

class BackendDev(Employee):
    atri = "B"
        
class FullstackDev(FrontendDev, BackendDev):
    atri = "Fu"

In [ ]:
f = FullstackDev()

In [ ]:
f.atr

<br>

MRO neplatí ovšem **jen pro metody**.

Jeho znalost lze aplikovat také na vlastnosti jednotlivých tříd.

### Souhrn

---

<br>

Dědičnost ale stejně jako ostatní koncepty nesmí být **zneužívána**.

<br>

Zřetězené odkazování přestává být **zřetelné** a program se stává příliš komplexní.

<br>

Nejprve interpret vyhledává **metody nebo atributy** v aktuální třídě (potomkovi) a potom teprve začíná sledovat MRO.

<br>

### 🧠 CVIČENÍ 9 🧠, Vytvoř třídu `Auto` a doplň následující:

- Definuj třídu `Auto`, s metodou `__init__`, která potřebuje jen parametr `znacka`, `barva`,
- vytvoř getter a setter metody pro atribut `max_rychlost` (maximum 200 km/h) a `hmotnost` (maximum 2 tuny),
- vytvoř třídu `OsobniAuto`, která dědí (`Auto`) atributy `znacka`, `barva` a navíc potřebuje instanční atribut `mista_k_sezeni`,
- pro `OsobniAuto` vytvoř getter a setter, který omezuje `max_rychlost` na 300 km/h a hodnota je vedená jako celé číslo,
- vytvoř třídu `NakladniAuto`, která dědí (`Auto`) atributy `znacka`, `barva` a navíc potřebuje instanční atribut `nostnost`,
- pro `NakladniAuto` vytvoř getter a setter, který omezuje `max_rychlost` na 100 km/h a hodnota je vedená jako celé číslo.

In [ ]:
class Auto:
    def __init__(self, znacka: str, barva: str):
        self.znacka = znacka
        self.barva = barva
        
    @property
    def max_rychlost(self) -> int:
        return self.__max_rychlost
    
    @max_rychlost.setter
    def max_rychlost(self, hodnota: int):
        if isinstance(hodnota, int) and hodnota <= 200:
            self.__max_rychlost = hodnota
        else:
            print("Neodpovídající datový typ nebo hodnota <200 km/h")
            
            
class OsobniAuto(Auto):
    
    def __init__(self, znacka: str, barva: str, mista_k_sezeni: int):
        super().__init__(znacka, barva)
        self.mista_k_sezeni = mista_k_sezeni
        
    @property
    def max_rychlost(self) -> int:
        return self.__max_rychlost
    
    @max_rychlost.setter
    def max_rychlost(self, hodnota: int):
        if isinstance(hodnota, int) and hodnota <= 300:
            self.__max_rychlost = hodnota
        else:
            print("Neodpovídající datový typ nebo hodnota <300 km/h")

In [ ]:
automobil_1 = Auto("BMW", "modrá")
automobil_2 = OsobniAuto("Audi", "bílá", 5)

In [ ]:
automobil_1.max_rychlost = 200

In [ ]:
automobil_1.__dict__

In [ ]:
automobil_2.max_rychlost = 300

In [ ]:
automobil_2.__dict__

In [ ]:
# tranzit = NakladniAuto("Ford", "šedá", 1.5)

In [ ]:
tranzit.max_rychlost = 99

<details>
    <summary>▶️ Řešení</summary>
    
    ```
    class Auto:
    def __init__(self, znacka: str, barva: str):
        self.znacka = znacka
        self.barva = barva
        self.__hmotnost = 0
        self.__max_rychlost = 0
    
    @property
    def hmotnost(self):
        return self.__hmotnost

    @hmotnost.setter
    def hmotnost(self, hodnota: int) -> None:
        if isinstance(hodnota, int) and 0 < hodnota < 2:
            self.__hmotnost = hodnota
        else:
            raise Exception("Špatný datový typ, nebo hodnota (<2 tun)")
    
    @property
    def max_rychlost(self):
        return self.__max_rychlost

    @max_rychlost.setter
    def max_rychlost(self, hodnota: float) -> None:
        if isinstance(hodnota, int) and 0 < hodnota < 200:
            self.__max_rychlost = hodnota
        else:
            raise Exception("Špatný datový typ, nebo hodnota (<200 km/h)")


    class OsobniAuto(Auto):
        def __init__(self, znacka: str, barva: str, mista_k_sezeni: int):
            super().__init__(znacka, barva)
            self.mista_k_sezeni = mista_k_sezeni

        @property
        def max_rychlost(self):
            return self.__max_rychlost

        @max_rychlost.setter
        def max_rychlost(self, hodnota: float) -> None:
            if isinstance(hodnota, int) and 0 < hodnota < 300:
                self.__max_rychlost = hodnota
            else:
                raise Exception("Špatný datový typ, nebo hodnota (<300 km/h)")


    class NakladniAuto(Auto):
        def __init__(self, znacka: str, barva: str, nostnost: float):
            super().__init__(znacka, barva)
            self.nostnost = nostnost

        @property
        def max_rychlost(self):
            return self.__max_rychlost

        @max_rychlost.setter
        def max_rychlost(self, hodnota: int) -> None:
            if isinstance(hodnota, int) and 0 < hodnota < 100:
                self.__max_rychlost = hodnota
            else:
                raise Exception("Špatný datový typ, nebo hodnota (<100 km/h)")
    ```
</details>

<br>

## Abstrakce

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.CFwkjtcCFee-YqXbmNDmxAHaHa%26pid%3DApi&f=1&ipt=b4737f334dd002ab30b8da0699a297e413334fd0ea059deb6cf4a6a0087b3f6b&ipo=images" width="300" style="margin-left:auto; margin-right:auto">

### Význam pojmu

---

Abstrakce je o redukci množství detailů.

V OOP stojí na tom, že uživatel nemusí znát všechny implementace, ale pouze jména objektů.

<br>

Stejně jako v praxi nevíš, co se všechno děje, pokud na chytrém telefonu zmáčkneš tlačítko pro vytvoření fotky.

Tak ani v Pythonu nepotřebuješ vědět všechny detaily:

In [ ]:
"matous".title()

In [ ]:
"OOP".lower()

### V praxi

---

<br>

Podobně můžeš vidět *abstrakci* v OOP.

Jak autor svojí knihovny se snažíš usnadnit práci ostatní uživatelům knihovny.

Proto jim stačí vytvoření přehledného atributu `typ_uctu`, který už si sám dohledá a zavolá konkrétní funkce/metody.

<!-- 
```python
class GooglePaymentProcessor:
    
    def __init__(self, order: int):
        self.order = order

    def pay(self):
        print(
            f"Nr.order: {self.order}",
            "Processing GooglePay..",
            "Verifying security code..",
            "Changing order status..",
            "-" * 25,
            sep="\n"
        )
```

```python
order_1 = GooglePaymentProcessor("1234567890")
order_1.pay()
``` -->

In [ ]:
class Uzivatel:
    def __init__(self, jmeno: str, email: str):
        self.email = email
        self.jmeno = jmeno

    @property
    def typ_uctu(self):
        return self.__typ_uctu
    
    @typ_uctu.setter
    def typ_uctu(self, typ: str):
        if isinstance(typ, str) \
            and typ.title() in ("Gold", "Platinum", "Diamond"):
            self.__typ_uctu = typ
            print("Chystám nový účet...")
        else:
            raise Exception()

In [ ]:
user_1 = Uzivatel("Matouš", "matous@gmail.com")

In [ ]:
user_1.typ_uctu = "gold"

<br>

Jako uživatel takové knihovny, potom neřeším implementaci těchto pomocných funkcí.

Nemusím je testovat, ani dokumentovat, protože to je starost autora knihovny.

Stačí knihovnu a objekt aplikovat pro svoje účely.

### Knihovna abc

---

In [ ]:
from abc import ABC, abstractmethod

<br>

Pro chystání užitečnějších abstrakcí můžeš pracovat s knihovnou `abc`:

In [ ]:
class Uzivatel(ABC):
    def __init__(self, jmeno: str, email: str):
        self.email = email
        self.jmeno = jmeno

    @abstractmethod
    def typ_uctu(self):
        """Zprocesuje mi nastavení správného typu účtu."""
        pass

Nyní potřebuješ vytvořit různé **typy účtů**.

<br>

Např. 3 různé předplatitelné scénaře typu:
1. **Gold**, (100 CZK/mo)
2. **Platinum**, (200 CZK/mo),
3. **Diamond**, (300 CZK/mo).

In [ ]:
class GoldUcet(Uzivatel):
    CENA_GOLD_UCTU: int = 100
        
    def typ_uctu(self):
        print(f"Uživatel: {self.jmeno} zvolil účet Gold, cena: {self.CENA_GOLD_UCTU}.")

In [ ]:
class PlatinumUcet(Uzivatel):
    CENA_PLATINUM_UCTU: int = 200
        
    def typ_uctu(self):
        print( f"Uživatel: {self.jmeno} zvolil účet Platinum, cena: {self.CENA_PLATINUM_UCTU}.")

In [ ]:
class DiamondUcet(Uzivatel):
    CENA_DIAMOND_UCTU: int = 300
        
    def typ_uctu(self):
        print( f"Uživatel: {self.jmeno} zvolil účet Diamond, cena: {self.CENA_DIAMOND_UCTU}.")

In [ ]:
user_2 = GoldUcet("Lukáš", "lukas.gulas@email.cz")
user_2.typ_uctu()

In [ ]:
user_3 = PlatinumUcet("Jan", "jan.adam@email.cz")
user_3.typ_uctu()

In [ ]:
user_4 = DiamondUcet("Marek", "marek.honza@email.cz")
user_4.typ_uctu()

<br>

`ABC` je objekt, kterou musíš zdědit z modulu `abc`. Python defaultně nepracuje s konceptem **abstraktních tříd jako jiné jazyky**.

<br>

Dále musíš označit metodu jako abstraktní metodu. Použij dekorátor `@abstractmethod`.

<br>

V abstraktní metodě nepíšeš žádné ohlášení, pouze dokumentaci abstraktní metody a ohlášení `pass`.

Dále také nemůžeš tvořit instance pro abstraktní třídu. Složí pouze jako nějaký odkaz.

<br>

### 🧠 CVIČENÍ 10 🧠, Vytvoř třídu `HerniPostava` a doplň následující:

- Definuj abstraktní třídu `HerniPostava`, která dědí objekt `ABC` z knihovny `abc`,
- třída`HerniPostava` pracuje s instanční atributem `_zivoty` s hodnotou `100`,
- třída`HerniPostava` pracuje se dvěma abstraktními metodami `utok` a `obrana`,
- definuj třídu `Rytir`, který dědí od třídy `HerniPostava` a pracuje s vlastním atributem `_zivoty` s hodnotou `150`,
- definuj metodu `utok` pro třídu `Rytir` s vypisovanou zprávou:`Rytíř útočí mečem!`,
- definuj metodu `obrana` pro třídu `Rytir` s vypisovanou zprávou:`Rytíř se brání štítem!`,
- definuj třídu `Mag`, který dědí od třídy `HerniPostava` a pracuje s vlastním atributem `_zivoty` s hodnotou `80`,
- definuj metodu `utok` pro třídu `Mag` s vypisovanou zprávou:`Mag útočí kouzlem!`,
- definuj metodu `obrana` pro třídu `Mag` s vypisovanou zprávou:`Mag se brání magickým štítem!`.

In [ ]:
matous_rytir = Rytir()
lukas_mag = Mag()

<details>
    <summary>▶️ Řešení</summary>
    
    ```
    from abc import ABC, abstractmethod


    class HerniPostava(ABC):
        def __init__(self):
            self._zivoty = 100

        @abstractmethod
        def utok(self):
            pass

        @abstractmethod
        def obrana(self):
            pass


    class Rytir(HerniPostava):
        def __init__(self):
            super().__init__()
            self._zivoty = 150

        def utok(self):
            print("Rytíř útočí mečem!")

        def obrana(self):
            print("Rytíř se brání štítem!")


    class Mag(HerniPostava):
        def __init__(self):
            super().__init__()
            self._zivoty = 80

        def utok(self):
            print("Mag útočí kouzlem!")

        def obrana(self):
            print("Mag se brání magickým štítem!")
    ```
</details>

---